<a href="https://colab.research.google.com/github/ahmedyahia01/ComfyUI-Colab/blob/main/ComfyUI_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title خلية التحقق من وجود GPU
!nvidia-smi

Mon Aug 11 08:07:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:

# @title الخلية 1: التحقق من المفاتيح السرية
import os
from google.colab import userdata

print("--- التحقق من المفاتيح السرية ---")
# التحقق من وجود المفاتيح المطلوبة في Colab Secrets
try:
    hf_token = userdata.get('HF_TOKEN')
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    if not hf_token or not ngrok_token:
        raise ValueError("أحد المفاتيح (HF_TOKEN أو NGROK_AUTH_TOKEN) غير موجود.")
    print("✅ تم العثور على جميع المفاتيح السرية المطلوبة.")
except Exception as e:
    print(f"❌ خطأ: {e}")
    print("   الرجاء التأكد من إضافة HF_TOKEN و NGROK_AUTH_TOKEN في قسم الأسرار (🔑).")

--- التحقق من المفاتيح السرية ---
✅ تم العثور على جميع المفاتيح السرية المطلوبة.


In [2]:

# @title الخلية 2: التثبيت الشامل والنهائي (على بيئة Colab المؤقتة)
import os
from huggingface_hub import hf_hub_download
from google.colab import userdata

# --- 1/4: تثبيت ComfyUI بشكل كامل ---
print("--- 1/4: جاري تحميل وتثبيت نسخة كاملة من ComfyUI...")
COMFYUI_DIR = "/content/ComfyUI"
get_ipython().system(f'git clone --recurse-submodules https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"')
print("✅ تم تثبيت ComfyUI بشكل كامل وسليم.")

# --- 2/4: تثبيت المتطلبات البرمجية ---
print("\n--- 2/4: تثبيت كل المتطلبات البرمجية...")
%cd $COMFYUI_DIR
print("   - إزالة xformers من المتطلبات لتجنب التعارض...")
get_ipython().system(f"sed -i '/xformers/d' requirements.txt")
print("   - تثبيت المتطلبات الأساسية...")
get_ipython().system('pip install -r requirements.txt --quiet')
print("   - تثبيت مكتبة uv الإضافية...")
get_ipython().system('pip install uv --quiet')
print("✅ تم تثبيت كل المتطلبات بنجاح.")

# --- 3/4: تثبيت الإضافات (Custom Nodes) ---
print("\n--- 3/4: تثبيت الإضافات...")
nodes_dir = os.path.join(COMFYUI_DIR, "custom_nodes")
%cd $nodes_dir
get_ipython().system('git clone https://github.com/ltdrdata/ComfyUI-Manager.git --quiet')
get_ipython().system('git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git --quiet')
manager_req = os.path.join(nodes_dir, "ComfyUI-Manager", "requirements.txt")
if os.path.exists(manager_req):
    get_ipython().system(f'pip install -r {manager_req} --quiet')
print("✅ تم تثبيت الإضافات بنجاح.")

# --- 4/4: تحميل النموذج الأساسي ---
print("\n--- 4/4: تحميل النموذج الأساسي (هذه الخطوة قد تستغرق وقتاً طويلاً)...")
checkpoints_dir = os.path.join(COMFYUI_DIR, "models", "checkpoints")
os.makedirs(checkpoints_dir, exist_ok=True)
hf_token = userdata.get('HF_TOKEN')
try:
    hf_hub_download(
        repo_id="stabilityai/stable-diffusion-3-medium",
        filename="sd3_medium_incl_clips.safetensors",
        local_dir=checkpoints_dir,
        local_dir_use_symlinks=False,
        token=hf_token
    )
    print("   - ✅ تم تحميل النموذج بنجاح.")
except Exception as e:
    print(f"   - ❌ فشل تحميل النموذج. الخطأ: {e}")

print("\n🎉🎉🎉 اكتمل الإعداد الكامل والنهائي! 🎉🎉🎉")

--- 1/4: جاري تحميل وتثبيت نسخة كاملة من ComfyUI...
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 22905, done.
remote: Total 22905 (delta 0), reused 0 (delta 0), pack-reused 22905 (from 1)
Receiving objects: 100% (22905/22905), 72.04 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (15367/15367), done.
✅ تم تثبيت ComfyUI بشكل كامل وسليم.

--- 2/4: تثبيت كل المتطلبات البرمجية...
/content/ComfyUI
   - إزالة xformers من المتطلبات لتجنب التعارض...
   - تثبيت المتطلبات الأساسية...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


In [3]:

# @title الخلية 3: المشغّل السريع (بدون Drive)
import os
import time
import re
from urllib import request, error
from google.colab import userdata

def wait_for_service(port, timeout=180): # 3 دقائق كافية الآن
    print(f"⏳ جاري انتظار الخادم ليكون جاهزًا...", end="")
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            with request.urlopen(f"http://localhost:{port}/", timeout=1):
                print("\n✅ الخادم يعمل الآن وجاهز للاستقبال.")
                return True
        except (error.URLError, ConnectionRefusedError):
            print(".", end="", flush=True)
            time.sleep(2)
    print("\n❌ فشل تشغيل الخادم في الوقت المحدد.")
    return False

PORT = 8188
COMFYUI_DIR = "/content/ComfyUI"
LOG_FILE = "/content/comfyui_run.log"

print("🧹 خطوة التنظيف: التأكد من أن البورت فارغ...")
get_ipython().system(f'fuser -k {PORT}/tcp')
time.sleep(2)
print("✅ البورت الآن جاهز للاستخدام.")

print(f"🚀 جاري تشغيل ComfyUI في الخلفية...")
%cd $COMFYUI_DIR
get_ipython().system_raw(f'python main.py --listen 0.0.0.0 --port {PORT} --preview-method auto > {LOG_FILE} 2>&1 &')

if wait_for_service(PORT):
    print("\n🔗 جاري محاولة إنشاء رابط عام عبر Cloudflared...")
    if not os.path.exists("/content/cloudflared"):
        get_ipython().system('wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64')
        get_ipython().system('chmod +x /content/cloudflared')
    get_ipython().system_raw(f'/content/cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate > /content/cloudflared.log 2>&1 &')
    time.sleep(5)
    link = None
    try:
        with open("/content/cloudflared.log", "r") as f:
            content = f.read()
            match = re.search(r"https://[A-Za-z0-9_-]+\.trycloudflare\.com", content)
            if match:
                link = match.group(0)
    except FileNotFoundError: pass

    if link:
        print("\n" + "="*70)
        print("🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك. اضغط عليه لفتحه. 🎉🎉🎉")
        print(link)
        print("="*70)
    else: # Fallback to ngrok
        print("\n⚠️ فشل Cloudflared، جاري محاولة استخدام ngrok...")
        from pyngrok import ngrok
        ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
        ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(PORT)
        print("\n" + "="*70)
        print("🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك عبر ngrok. 🎉🎉🎉")
        print(public_url)
        print("="*70)
else:
    print("\n" + "="*70)
    print("‼️ فشل الخادم في البدء. جاري عرض سجل الأخطاء:")
    print("="*70)
    get_ipython().system(f'cat {LOG_FILE}')

🧹 خطوة التنظيف: التأكد من أن البورت فارغ...
✅ البورت الآن جاهز للاستخدام.
🚀 جاري تشغيل ComfyUI في الخلفية...
/content/ComfyUI
⏳ جاري انتظار الخادم ليكون جاهزًا...............
✅ الخادم يعمل الآن وجاهز للاستقبال.

🔗 جاري محاولة إنشاء رابط عام عبر Cloudflared...

🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك. اضغط عليه لفتحه. 🎉🎉🎉
https://scholarships-circus-consisting-performer.trycloudflare.com
